In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [11]:
#unión de los dos datasets que hemos sacado
df1 = pd.read_csv("data/initial_data/processed/books_enriched_full.csv") 
df2 = pd.read_csv("data/initial_data/processed/books_2020_to_2025.csv")
# Unir verticalmente (uno debajo del otro)
full_data = pd.concat([df1, df2], ignore_index=True)
# Guardar el resultado
full_data.to_csv("data/full_data/full_data_base.csv", index=False)
print("Archivos combinados correctamente en full_data_base.csv")

Archivos combinados correctamente en full_data_base.csv


In [13]:
df = pd.read_csv("data/full_data/full_data_base.csv")
df

C:\Users\elena\AppData\Local\Temp\ipykernel_6988\1207315263.py:1: DtypeWarning: Columns (0,1,2,3,5,6,7,8,9,10,11,12,13,14,15,16,18,20,21,24,25,26,27,28,30,31,32,33,36,37,38,39,40,44) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/full_data/full_data_base.csv")


,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,Ratings Count,Thumbnail,Info Link,Google ID,Genre,Fetched From,Format,Series,Physical Format,Release Date
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9780062369635,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,NaN,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,http://books.google.es/books?id=SzxlzgEACAAJ&d...,SzxlzgEACAAJ,yaoi,Google Books,NaN,NaN,NaN,2021-12-02
74785,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,http://books.google.com/books/content?id=Te04E...,http://books.google.es/books?id=Te04EQAAQBAJ&d...,Te04EQAAQBAJ,yaoi,Google Books,NaN,NaN,NaN,2025
74786,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,http://books.google.com/books/content?id=s_8_E...,http://books.google.es/books?id=s_8_EAAAQBAJ&d...,s_8_EAAAQBAJ,yaoi,Google Books,NaN,NaN,NaN,2021-05-04
74787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,http://books.google.com/books/content?id=_yjnE...,http://books.google.es/books?id=_yjnEAAAQBAJ&d...,_yjnEAAAQBAJ,isekai comedy,Google Books,NaN,NaN,NaN,2020-05-01


In [14]:
df.shape

(74789, 45)

In [16]:
import pandas as pd

# Cargar datasets
df1 = pd.read_csv("data/initial_data/processed/books_enriched_full.csv") 
df2 = pd.read_csv("data/initial_data/processed/books_2020_to_2025.csv")

# Renombrar columnas de df2 para que coincidan con df1 y aplicar lo que pediste
df2 = df2.rename(columns={
    'Title': 'title',
    'Author(s)': 'author',
    'Series': 'series',
    'Average Rating': 'rating',
    'Description': 'description',
    'Language': 'language',
    'ISBN': 'isbn',
    'Categories': 'genres',          # <- se convierte en 'genres' como en df1
    'Genre': 'subgénero',            # <- nuevo campo en español
    'Format': 'format_temp',         # <- nombre temporal para no sobrescribir
    'Physical Format': 'physical_format_temp',  # <- también temporal
    'Pages': 'pages',
    'Publisher': 'publisher',
    'Published': 'publishDate',
    'Release Date': 'firstPublishDate',
    'Ratings Count': 'numRatings',
    'Google ID': 'bookId'
})

# Fusionar 'format_temp' y 'physical_format_temp' en 'bookFormat'
df2['bookFormat'] = df2.apply(
    lambda row: ' - '.join(filter(None, [str(row.get('format_temp', '')).strip(), str(row.get('physical_format_temp', '')).strip()])),
    axis=1
)

# Eliminar columnas temporales
df2.drop(columns=['format_temp', 'physical_format_temp'], inplace=True, errors='ignore')

# Asegurar que ambas columnas 'genres' sean string
df1['genres'] = df1['genres'].fillna('').astype(str)
df2['genres'] = df2['genres'].fillna('').astype(str)

# Añadir columnas exclusivas de df1 a df2 con NaN
cols_only_in_df1 = ['awards', 'setting', 'coverImg', 'likedPercent']
for col in cols_only_in_df1:
    if col not in df2.columns:
        df2[col] = pd.NA

# Añadir columna 'subgénero' a df1 si no existe
if 'subgénero' not in df1.columns:
    df1['subgénero'] = pd.NA

# Igualar columnas en ambos DataFrames (añadir faltantes como NaN)
for col in df2.columns:
    if col not in df1.columns:
        df1[col] = pd.NA
for col in df1.columns:
    if col not in df2.columns:
        df2[col] = pd.NA

# Reordenar columnas según df1
final_columns = df1.columns.tolist()
df2 = df2.reindex(columns=final_columns)

# Concatenar ambos datasets
full_data = pd.concat([df1, df2], ignore_index=True)

# Guardar resultado final
full_data.to_csv("data/full_data/data.csv", index=False)

print("Archivos combinados correctamente con todos los ajustes aplicados")

C:\Users\elena\AppData\Local\Temp\ipykernel_6988\2302693096.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_data = pd.concat([df1, df2], ignore_index=True)


Archivos combinados correctamente con todos los ajustes aplicados


In [22]:
df_data = pd.read_csv('data/full_data/data.csv')
df_data

C:\Users\elena\AppData\Local\Temp\ipykernel_6988\3764728690.py:1: DtypeWarning: Columns (2,9,11,12,16,18,20,21,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv('data/full_data/data.csv')


,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,likedPercent,setting,coverImg,bbeScore,bbeVotes,price,subgénero,Thumbnail,Info Link,Fetched From
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816.0,30516.0,5.09,NaN,NaN,NaN,NaN
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233.0,26923.0,7.38,NaN,NaN,NaN,NaN
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9780062369635,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402.0,23328.0,NaN,NaN,NaN,NaN,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,NaN,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116.0,20452.0,NaN,NaN,NaN,NaN,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448.0,14874.0,2.1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74784,SzxlzgEACAAJ,Afterimage Slow Motion,NaN,Jyanome,NaN,NaN,en,3964335037,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,yaoi,NaN,http://books.google.es/books?id=SzxlzgEACAAJ&d...,Google Books
74785,Te04EQAAQBAJ,When Fate Holds Its Breath,NaN,Takiba,NaN,NaN,de,9783551624680,Comics & Graphic Novels,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,yaoi,http://books.google.com/books/content?id=Te04E...,http://books.google.es/books?id=Te04EQAAQBAJ&d...,Google Books
74786,s_8_EAAAQBAJ,Midnight Delivery Sex,NaN,Neneko Narazaki,NaN,Mit SNS-Card zum Sammeln in der ersten Auflage...,de,9783551620668,Comics & Graphic Novels,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,yaoi,http://books.google.com/books/content?id=s_8_E...,http://books.google.es/books?id=s_8_EAAAQBAJ&d...,Google Books
74787,_yjnEAAAQBAJ,Monster Girl: 2,NaN,Kazuki Funatsu,NaN,"Dopo il loro incontro, Yatsuki si ritrova ad a...",it,9788834915899,Comics & Graphic Novels,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,isekai comedy,http://books.google.com/books/content?id=_yjnE...,http://books.google.es/books?id=_yjnEAAAQBAJ&d...,Google Books


In [23]:
# Unir columnas 'genres' y 'subgénero' (si existen ambas)
if 'genres' in df_data.columns and 'subgénero' in df_data.columns:
    df_data['genres'] = df_data.apply(
        lambda row: ' - '.join(filter(None, [str(row['genres']).strip(), str(row['subgénero']).strip()])),
        axis=1
    )
    # Eliminar la columna 'subgénero' después de combinar
    df_data.drop(columns=['subgénero'], inplace=True)

    print("Columnas 'genres' y 'subgénero' combinadas correctamente.")
else:
    print("Una de las columnas 'genres' o 'subgénero' no existe.")

Columnas 'genres' y 'subgénero' combinadas correctamente.


In [24]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74789 entries, 0 to 74788
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            74789 non-null  object 
 1   title             74789 non-null  object 
 2   series            23470 non-null  object 
 3   author            74517 non-null  object 
 4   rating            53113 non-null  float64
 5   description       71379 non-null  object 
 6   language          73542 non-null  object 
 7   isbn              74028 non-null  object 
 8   genres            74789 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        73316 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             73162 non-null  object 
 13  publisher         66657 non-null  object 
 14  publishDate       74420 non-null  object 
 15  firstPublishDate  53463 non-null  object 
 16  awards            52478 non-null  object

In [26]:
df_data.to_csv("data/full_data/df_data.csv", index=False)